This notebook contains all the code needed to reproduce the results from the thesis "Voting systems at the Eurovision Song Contest", authored my me (Lennart Beekhuis, University of Amsterdam) and supervised by Arthur Boixel and Ulle Endriss.

## Contents

1. Object-oriented classes
2. Importing data into the class format
3. Contest class and importing data for post-2015 contests
4. Applying different rules to an ESC
5. Finding a score vector which makes a certain country win
6. Determining the effect of collusion on a country's rankings

## Object-oriented classes

First, a Contest class was made. The Contest class contains other class objects: Country objects, and one Rule object. To compute a score, it loops over all the Country objects, takes their rankings, applies the specified scoring vector on it and adds the scores up. Using this, we can apply a different rule to an ESC than the original rule used.

In [1]:
import os
import random

class Country():
    """Country class which specifies the ranking of a country and its abbreviation."""
    def __init__(self, country, ranking_list, semifinal_score=0):
        self.country = country
        self.ranking_list = ranking_list
        
    def get_country_name(self):
        return self.country

    def get_ranking_list(self):
        return self.ranking_list

    def get_ith_ranking(self, i):
        return self.ranking_list[i-1]
    
class Rule():
    """Rule class consisting of a scoring vector and a vector length.
    Scoring vector is specified from largest to smallest score."""
    def __init__(self, scoring_vector, vector_length):
        self.scoring_vector = scoring_vector
        self.vector_length = vector_length

    def get_scoring_vector(self):
        return self.scoring_vector

    def get_vector_length(self):
        return self.vector_length

class Contest():
    """Contest class includes everything to compute the original outcome
    of a certain year's ESC. Later will also be able to solve the problem(s)
    specified in the thesis proposal."""
    def __init__(self, country_list, voting_rule, nr_of_semifinals=0):
        self.country_list = country_list
        self.removed_list = []
        self.voting_rule = voting_rule
        self.nr_of_countries = len(country_list)
        self.nr_of_semifinals = nr_of_semifinals
        
    def compute_result(self, rule=None):
        """Outputs a dict with as key the country and as value the final score
        of that country.
        If a rule is specified, computes the result using that rule instead of
        the rule specified in the Contest object."""
        result = dict()
        if rule is None:
            rule_vector = self.voting_rule.get_scoring_vector()
            rule_length = self.voting_rule.get_vector_length()
        else:
            rule_vector = rule.get_scoring_vector()
            rule_length = rule.get_vector_length()
        for country in self.country_list:
            # this part of the code is used for contests after 2015.
            # They use a different class which will be specified later.
            if isinstance(country, Country2015) and rule is None:
                jury_list = country.get_ranking_list_jury()
                tele_list = country.get_ranking_list_tele()

                rank = 0
                for c in jury_list:
                    if rank + 1 > rule_length:
                        break
                    if c not in result:
                        result[c] = 0
                    result[c] += rule_vector[rank]
                    rank += 1

                rank = 0
                for c in tele_list:
                    if rank + 1 > rule_length:
                        break
                    if c not in result:
                        result[c] = 0
                    result[c] += rule_vector[rank]
                    rank += 1
            else:
                ranking_list = country.get_ranking_list()
                rank = 0
                for c in ranking_list:
                    if rank + 1 > rule_length:
                        break
                    if c not in result:
                        result[c] = 0
                    result[c] += rule_vector[rank]
                    rank += 1

        return result

    def print_result(self, rule=None):
        """Prints the result in a ranking list format."""
        result = self.compute_result(rule)
        for element in sorted(result, key=result.get, reverse=True):
            print(element, result[element])
        print()

## Importing data into the class format

Before we can compute anything, we need to import the data first. Make sure the 'votes.csv' is in the same directory as this notebook, otherwise import_scores_linear_only() won't work.

In [2]:
import csv

def import_scores_linear_only(first_year, last_year, file_name = 'votes.csv'):
    """ HARDCODED FUNCTION
    Reads and puts the rankings of each country in the finals
    from first_year to last_year in a dictionary. This dict has 2 levels:
    Top level: key = year, value = dict of participating countries
    Bottom level: key = country, value = ranked list of preferences
    NOTE: this function only works for linear rankings, and doesn't work
    with post-2015 rankings."""
    f = open(file_name, "r")
    ff = csv.reader(f)

    ESC = dict()
    for year in range(first_year, last_year):
        ESC[str(year)] = dict()

    for row in ff:
        if row[0] == 'year':
            continue
        if int(row[0]) not in range(first_year, last_year) or row[1] != 'final' or row[6] == '0':
            continue
        if row[2] not in ESC[row[0]]:
            ESC[row[0]][row[2]] = dict()
        ESC[row[0]][row[2]][row[3]] = int(row[6])
        
    scores_to_rankings(ESC)
    return ESC

def scores_to_rankings(ESC_dict):
    """This is a helper function for import_scores_linear_only.
    It turns lists of (country, score) tuples into a ranked list of preferences."""
    for year in ESC_dict:
        for country in ESC_dict[year]:
            score_dict = ESC_dict[year][country]
            ranking_list = []
            for scored_country in sorted(score_dict, key=score_dict.get, reverse=True):
                ranking_list.append(scored_country)
            ESC_dict[year][country] = ranking_list
    pass

def dict_to_classes(d, scoring_rule):
    """Takes as input a dict outputted by import_scores_linear_only() and a scoring rule,
    and returns a dict with objects of class Contest."""
    ESC_objects = dict()
    for year in range(1975,2016):
        ESC_objects[str(year)] = None
        
    for year in d:
        country_list = []
        for country in d[year]:
            new_country = Country(country, d[year][country])
            country_list.append(new_country)
        ESC_objects[year] = Contest(country_list, scoring_rule)
    
    return ESC_objects

## Contest class and importing data for post-2015 contests

The classes and functions specified only work for the contests until 2015. After 2015, the vote was split into two parts: a jury vote and a televote. To work with these contests, another Country class was made which inherits all the functions from the older Country class, and a new import function was created.

Note: make sure 'votes.csv' is in the same directory as this notebook.

In [3]:
class Country2015(Country):
    """Class object which handles contests after 2015. Inherits original
    Country functions. To compute the outcome, the jury and televoting
    ranking lists are used.
    To apply older rules to these contests, the jury and televoting
    ranking lists are combined into one ranking list using pre_2015_ify()"""
    def __init__(self, country, jury, tele, total, semifinal_score=0):
        self.country = country
        self.ranking_list_jury = jury
        self.ranking_list_tele = tele
        self.ranking_list = pre_2015_ify(jury, tele, total)
        self.semifinal_score = semifinal_score
        
    def get_ranking_list_jury(self):
        return self.ranking_list_jury
    
    def get_ranking_list_tele(self):
        return self.ranking_list_tele

def import_scores_2015_and_beyond(file_name = 'votes.csv'):
    """ HARDCODED FUNCTION
    Reads and puts the rankings of each country in the finals
    from 2016 to 2019 in three dictionaries. This dict has 2 levels:
    Top level: key = year, value = dict of participating countries
    Bottom level: key = country, value = ranked list of preferences
    ESC_jury and ESC_tele contain a linear ranking,
    ESC_total contains (country, score) tuples, obtained by
    adding up the jury score and the televoting score."""
    f = open(file_name, "r")
    ff = csv.reader(f)

    ESC_jury, ESC_tele, ESC_total = dict(), dict(), dict()

    for year in range(2016,2020):
        ESC_jury[str(year)], ESC_tele[str(year)] = dict(), dict()
        ESC_total[str(year)] = dict()

    for row in ff:
        if row[0] == 'year':
            continue
        if int(row[0]) not in range(2016,2020) or row[1] != 'final' or row[6] == '0':
            continue
        if row[2] not in ESC_jury[row[0]] and row[8] != 0:
            ESC_jury[row[0]][row[2]] = dict()
        if row[2] not in ESC_tele[row[0]] and row[7] != 0:
            ESC_tele[row[0]][row[2]] = dict()
        if row[2] not in ESC_total[row[0]]:
            ESC_total[row[0]][row[2]] = dict()
        if row[8] != 0:
            ESC_jury[row[0]][row[2]][row[3]] = int(row[8])
        if row[7] != 0:
            ESC_tele[row[0]][row[2]][row[3]] = int(row[7])
        ESC_total[row[0]][row[2]][row[3]] = int(row[6])

    scores_to_rankings(ESC_jury)
    scores_to_rankings(ESC_tele)

    return ESC_jury, ESC_tele, ESC_total

def pre_2015_ify(jury, tele, total, voting_rule=Rule([12,10,8,7,6,5,4,3,2,1], 10)):
    """Converts the post 2015 total score to a 1975-2012 ranking.
    To break ties, the jury vote is checked: the country which is
    ranked higher in the jury vote wins the tie."""
    result = dict()
    rule_vector = voting_rule.get_scoring_vector()
    rule_length = voting_rule.get_vector_length()
    
    def swap_positions(l, elem1, elem2):
        pos1, pos2 = l.index(elem1), l.index(elem2)
        l[pos1], l[pos2] = l[pos2], l[pos1] 
    
    def bubble_sort_tuple_list(l):
        for i in range(len(l) - 1):
            if l[i][1] == l[i + 1][1]:
                if tele.index(l[i+1][0]) < tele.index(l[i][0]):
                    swap_positions(tuple_list, tuple_list[i+1], tuple_list[i])
    
    pre_2015_ranking = []
    tuple_list = []
    
    for element in sorted(total, key=total.get, reverse=True):
        tuple_list.append((element, total[element]))
        
    for i in range(5):
        bubble_sort_tuple_list(tuple_list)
    
    ranking_list = []
    for i in range(len(tuple_list)):
        ranking_list.append(tuple_list[i][0])
        
    return ranking_list
        
def dicts_to_2015_classes(d_jury, d_tele, d_total, voting_rule):
    """Takes as input a jury and televoting dict outputted by
    import_scores_2015_and_beyond() and a scoring rule, 
    and returns a dict with objects of class Contest."""
    ESC_classified = dict()
    for year in range(2016,2020):
        ESC_classified[str(year)] = None

    for year in d_jury:
        country_list = []
        for country in d_jury[year]:
            new_country = Country2015(country,
                                      d_jury[year][country], 
                                      d_tele[year][country],
                                      d_total[year][country])
            country_list.append(new_country)
        ESC_classified[year] = Contest(country_list, voting_rule)

    return ESC_classified


## Applying different rules to an ESC

Everything is now in place to apply a different rule to ESCs. Below is some code which imports all relevant ESCs into dicts, prints results using the original rule, and prints results using a different rule.
These commands were used to acquire the results in the rules_and_winners.xlsx file. The table in that file is also included in the thesis itself.

In [4]:
# rules
rule_1962 = Rule([3,2,1], 3)
rule_1963 = Rule([5,4,3,2,1], 5)
rule_1964_66 = Rule([5,3,1], 3)
rule_1975_2015 = Rule([12,10,8,7,6,5,4,3,2,1], 10)

# dicts with contests
ESC_1962_66 = dict_to_classes(import_scores_linear_only(1962, 1967), rule_1964_66)
ESC_1962_66['1962'].voting_rule = Rule([3,2,1], 3)
ESC_1962_66['1963'].voting_rule = Rule([5,4,3,2,1], 5)

ESC_1975_2015 = dict_to_classes(import_scores_linear_only(1975, 2016), rule_1975_2015)

d_jury, d_tele, d_total = import_scores_2015_and_beyond()
ESC_2016_19 = dicts_to_2015_classes(d_jury, d_tele, d_total, rule_1975_2015)

# print original result
ESC_1962_66['1962'].print_result()
ESC_1975_2015['1975'].print_result()
ESC_2016_19['2016'].print_result()

# print result when using a different rule
ESC_1962_66['1962'].print_result(rule_1964_66)
ESC_1975_2015['1975'].print_result(rule_1963)
ESC_2016_19['2016'].print_result(rule_1962)

# This function was used to help construct the rules_and_winners.xlsx table
def print_result_multiple_ESCs(ESC_dict, rule):
    """Takes as input a dict of ESCs and a rule.
    Outputs the result for all the ESCs in the dict using the rule."""
    
    for contest in ESC_dict:
        print("### " + str(contest) + " ###")
        ESC_dict[contest].print_result(rule)
        
# print_result_multiple_ESCs(ESC_1975_2015, rule_1962)


fr 26
mc 13
lu 11
yu 10
gb 10
de 9
se 4
fi 4
it 3
no 2
ch 2
dk 2

nl 152
gb 138
it 115
fr 91
lu 84
ch 77
fi 74
se 72
ie 68
es 53
il 40
mt 32
yu 22
mc 22
be 17
pt 16
de 15
no 11
tr 3

ua 534
au 511
ru 491
bg 307
se 261
fr 257
am 249
pl 229
lt 200
be 181
mt 153
nl 153
at 151
il 135
lv 132
it 124
az 117
rs 115
hu 108
ge 104
cy 96
es 77
hr 73
gb 62
cz 41
de 11

fr 40
mc 20
lu 17
yu 15
gb 15
de 13
se 6
fi 6
it 4
no 3
ch 3
dk 2

nl 54
gb 46
it 34
fr 31
fi 21
lu 20
ch 18
se 16
ie 14
es 6
il 6
mt 5
pt 5
de 3
be 2
no 2
yu 2

ua 45
au 41
ru 30
se 21
bg 18
fr 15
am 14
rs 13
lt 10
lv 8
be 7
at 7
az 6
it 4
cy 3
ge 2
nl 2
hu 2
pl 1
hr 1
es 1
mt 1



## Finding a score vector which makes a certain country win

To find the score vector which makes country X win, the mystic package was used. The mystic framework provides a collection of optimization algorithms and tools that allows the user to more robustly (and easily) solve hard optimization problems. (source: https://pypi.org/project/mystic/)
To use mystic, download it by using pip3 or the provided link.

A system of inequalities is first created in the format specified by mystic. This system is then solved using the mystic solver. Note that this solver only outputs one solution. If you run the solver again, it may output a different solution.

In [6]:
import mystic.symbolic as ms

class Contest(Contest):
    
    def rankings_to_variable_lists(self):
        """Loops over all the rankings. For each ranked country x, add
        the corresponding variable to their dictionary value.
        f.e. for country X ranked 1st by country Y, add the variable A
        corresponding to 1st place to their dictionary value.
        Returns a dictionary, key = country, value = list of variables."""
        result = dict()
        mapping = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
        # A = 1st, B = 2nd, ..., J = 10th
        
        for country in self.country_list:
            result[country.get_country_name()] = []
        for country in self.country_list:
            rank = 0
            for elem in country.get_ranking_list():
                result[elem].append(mapping[rank])
                rank += 1
                
        return result
    
def reimport_data():
    """This function needs to be run after the Contest class has been redefined.
    It reimports all the data using the new contest class so that new functions
    can be used."""
    ESC_1962_66 = dict_to_classes(import_scores_linear_only(1962, 1967), rule_1964_66)
    ESC_1962_66['1962'].voting_rule = Rule([3,2,1], 3)
    ESC_1962_66['1963'].voting_rule = Rule([5,4,3,2,1], 5)

    ESC_1975_2015 = dict_to_classes(import_scores_linear_only(1975, 2016), rule_1975_2015)

    d_jury, d_tele, d_total = import_scores_2015_and_beyond()
    ESC_2016_19 = dicts_to_2015_classes(d_jury, d_tele, d_total, rule_1975_2015)
    
    return ESC_1962_66, ESC_1975_2015, ESC_2016_19
    
def make_equation(alphabet_list):
    """Takes as input a list from a dict provided by rankings_to_variable_lists,
    and outputs an equation which is compatible with mystic."""
    equation = ''
    mapping = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
    for letter in mapping:
        count = alphabet_list.count(letter)
        if count > 0:
            if count == 1:
                equation += letter + ' + '
            else:
                equation += str(count) + '*' + letter + ' + '
    
    return equation[:-3]
            
def make_mystic_string(letter_dict, new_winner):
    """Takes as input a dict provided by rankings_to_variable_lists and a winner,
    and outputs a system of inequalities (string) which mystic can solve."""
    
    mystic_string = 'A > B\nB > C\nC > D\nD > E\nE > F\nF > G\nG > H\nH > I\nI > J\n'
    # Starting string makes sure each number in the scoring vector
    # is higher than the previous one
    first_part = make_equation(letter_dict[new_winner]) + ' > '
    for country in letter_dict:
        if country == new_winner:
            continue
        second_part = make_equation(letter_dict[country]) + '\n'
        inequality = first_part + second_part
        mystic_string += inequality
    # Every other string added is of the form score(x) > score(y),
    # where y is any country other than x.
        
    return mystic_string[:-1]

def solve_inequalities(inequalities):
    """This function should solve a system of inequalities produced by make_mystic_string."""
    var = list('ABCDEFGHIJ')
    eqns = ms.simplify(inequalities, variables=var)
    print("Simplified equations: ", eqns)
    
    constraint = ms.generate_constraint(ms.generate_solvers(eqns, var))
    solution = constraint([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27])
    print("Solution: ", solution)
    
    solutionDict = dict(zip(var,solution))
    print("solutionDict: ", solutionDict)

ESC_1962_66, ESC_1975_2015, ESC_2016_19 = reimport_data()
letter_dict = ESC_1975_2015['1975'].rankings_to_variable_lists()
solve_inequalities(make_mystic_string(letter_dict, 'gb'))

Simplified equations:  B > -3*A/4 - C/2 - D/2 - F/2 - G/4 - H/2 + I/2
A > -3*B/4 - C/2 - D/2 + E/2 - F/4 - G/4 - H/4 + I/2 + 3*J/2
E > F
A > -B - C/2 - D/4 - F/2 - G/4 - H/2 + I/2
F > G
E < 3*A/2 + 3*B/2 + C/2 + H - 3*I/2 - J/2
C > D
A > -B - C/2 - D/4 - F/4 - H/4 + I/4 + J/4
A > -B - C/4 - D/2 - F/2 - H/4
A > -B - C/2 - D/4 - F/4 - G/4 - H/4 + I/4
E < 3*A/2 + B + C/2 + D/2 - F/2
D > E
A > -B + D/4 + E - F/4 - G/4 - H/4 + I/4 + J/4
H > I
G > H
A > -B - C/2 - D/2 - F/2 - G/4 - H/4
A > -B - C/2 - D/2 - F/4 + 3*I/4 + J/4
A > -B - C/4 - D/4 - F/4 - H/2 + 3*I/4 + J/2
C > -3*A/2 - 3*B/2 - D/2 + 3*E/2 - F + 3*G/2 - H/2 + J
E < -2*A + B - C + D + F + H - J
A > -B - C/4 - D/4 + E/4 + 3*G/4
D > -A - 3*B/2 + E/2 - H/2 + J/2
B > -A/2 + 3*C/4 + D/4 - F/2 - G/4 - H/4 + I/4 + J/4
B > C
I > J
C > -3*A/2 + B - D/2 + 3*E/2 - F/2 + G/2 - H/2 + J
A > B
Solution:  [11.750000000000012, 4.000000000000011, 4.000000000000005, 6.000000000000014, -13.50000000000001, 7.000000000000008, 9.000000000000021, 9.000000

## Determining the effect of collusion on a country's rankings

To determine the effect of collusion on the ranking of a country, we remove the colluding countries from a contest then recompute the result. The recomputed result is compared to the original result to see how much the colluding countries have moved up or down. This is done for multiple contests, and the average is computed.

In [1]:
class Contest(Contest):
    
    def get_country_name_list(self):
        """Returns a list of names (abbreviations) of each participating country"""
        name_list = []
        for country in self.country_list:
            name_list.append(country.get_country_name())
        
        return name_list
    
    def remove_countries(self, countries):
        """Takes a list of country names and (re)moves them from the country list."""
        country_list = self.get_country_name_list()
        for name in countries:
            
            if name not in country_list:
                print('Country {} did not participate in this contest'.format(name))
                return None
            for country in self.country_list:
                if country.get_country_name() == name:
                    self.removed_list.append(country)
                    self.country_list.remove(country)
            
    def readd_countries(self):
        """Adds all removed countries back to country_list"""
        for i in range(len(self.removed_list)):
            elem = self.removed_list.pop()
            self.country_list.append(elem)
            

    
def compare_rankings(new, old):
    """Takes as input two dictionaries with rankings and computes
    the difference in placing for each country between the new ranking
    and the old ranking. Returns a dict with as key the country and as
    value the change between the old ranking and new ranking for that country."""
    result_changes = []

    new_ranking = []
    old_ranking = []
    for country in sorted(new, key=new.get, reverse=True):
        new_ranking.append(country)
    for country in sorted(old, key=old.get, reverse=True):
        old_ranking.append(country)

    rank = 0
    for country in new_ranking:
        result_changes.append(old_ranking.index(country) - rank)
        rank += 1

    return dict(zip(new_ranking, result_changes))

def check_collusion(ESC_dict, country_list):
    """Takes as input a dict with Contest objects and a list of countries,
    and returns a dict with as keys the countries and as values the average
    change of position for each country in these contests."""
    result = dict()
    
    for ESC in ESC_dict:
        old = ESC_dict[ESC].compute_result()
        ESC_dict[ESC].remove_countries(country_list)
        new = ESC_dict[ESC].compute_result()
        comparison = compare_rankings(new, old)
        for key in comparison:
            if key not in result:
                result[key] = 0
            result[key] += comparison[key]
    
    result = {k: v / total for total in (sum(result.values()),) for k, v in result.items()}
    # https://stackoverflow.com/questions/30964577/divide-each-python-dictionary-value-by-total-value
    return result

NameError: name 'Contest' is not defined

In [14]:
ESC_1962_66, ESC_1975_2015, ESC_2016_19 = reimport_data()
check_collusion(ESC_1975_2015, ['gr','cy'])


Country gr did not participate in this contest
Country cy did not participate in this contest
Country cy did not participate in this contest
Country cy did not participate in this contest
Country cy did not participate in this contest
Country cy did not participate in this contest
Country gr did not participate in this contest
Country gr did not participate in this contest
Country gr did not participate in this contest
Country cy did not participate in this contest
Country gr did not participate in this contest
Country gr did not participate in this contest
Country cy did not participate in this contest
Country cy did not participate in this contest


{'nl': 0.5,
 'gb': 1.0,
 'it': 0.0,
 'fr': 0.16666666666666666,
 'lu': 0.5,
 'ch': 0.16666666666666666,
 'fi': 0.16666666666666666,
 'se': 1.1666666666666667,
 'ie': 0.5,
 'es': -3.1666666666666665,
 'il': 2.1666666666666665,
 'mt': 0.16666666666666666,
 'yu': 0.5,
 'mc': -0.16666666666666666,
 'be': 0.6666666666666666,
 'pt': 0.6666666666666666,
 'de': -0.5,
 'no': 0.0,
 'tr': 0.3333333333333333,
 'at': -0.16666666666666666,
 'gr': -4.333333333333333,
 'dk': 0.16666666666666666,
 'ma': 0.0,
 'cy': -4.166666666666667,
 'is': 0.6666666666666666,
 'hr': 0.16666666666666666,
 'ba': 1.6666666666666667,
 'si': 0.5,
 'pl': 0.16666666666666666,
 'hu': 0.8333333333333334,
 'ru': 0.16666666666666666,
 'sk': 0.0,
 'ro': -0.8333333333333334,
 'ee': 1.0,
 'mk': 0.3333333333333333,
 'lt': 0.8333333333333334,
 'lv': 0.5,
 'ua': 0.3333333333333333,
 'cs': -0.16666666666666666,
 'al': -0.6666666666666666,
 'md': 0.16666666666666666,
 'am': -0.8333333333333334,
 'rs': 0.16666666666666666,
 'bg': 0.0,
 

In [41]:
import mystic.symbolic as ms

def make_equation(alphabet_list):
    """Takes as input a list from a dict provided by rankings_to_variable_lists,
    and outputs an equation which is compatible with mystic."""
    equation = ''
    mapping = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']
    for letter in mapping:
        count = alphabet_list.count(letter)
        if count > 0:
            if count == 1:
                equation += letter + ' + '
            else:
                equation += str(count) + '*' + letter + ' + '
    
    return equation[:-3]
            
def make_mystic_string(letter_dict, new_winner):
    """Takes as input a dict provided by rankings_to_variable_lists and a winner,
    and outputs a string which mystic can solve."""
    
    mystic_string = 'A > B\nB > C\nC > D\nD > E\nE > F\nF > G\nG > H\nH > I\nI > J\n'
    # Starting string makes sure each element of the score vector is higher than the prev
    first_part = make_equation(letter_dict[new_winner]) + ' > '
    for country in letter_dict:
        if country == new_winner:
            continue
        second_part = make_equation(letter_dict[country]) + '\n'
        inequality = first_part + second_part
        mystic_string += inequality
    # Every other string added is of the form score(x) > score(y),
    # where y is any country other than x.
        
    return mystic_string[:-1]

def solve_inequalities(inequalities):
    """This function should solve a system of inequalities produced by make_mystic_string."""
    var = list('ABCDEFGHIJ')
    eqns = ms.simplify(inequalities, variables=var)
    print("Simplified equations: ", eqns)
    
    constraint = ms.generate_constraint(ms.generate_solvers(eqns, var))
    solution = constraint([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27])
    print("Solution: ", solution)
    
    solutionDict = dict(zip(var,solution))
    print("solutionDict: ", solutionDict)


letter_dict = ESC_classified['1975'].rankings_to_variable_lists()
solve_inequalities(make_mystic_string(letter_dict, 'gb'))

Simplified equations:  A > B
A > -B - C/2 - D/4 - F/4 - H/4 + I/4 + J/4
B > C
F > G
A > -B - C/4 - D/2 - F/2 - H/4
A > -B - C/2 - D/2 - F/2 - G/4 - H/4
D > -A - 3*B/2 + E/2 - H/2 + J/2
A > -B - C/2 - D/4 - F/4 - G/4 - H/4 + I/4
C > D
A > -B - C/2 - D/2 - F/4 + 3*I/4 + J/4
A > -B + D/4 + E - F/4 - G/4 - H/4 + I/4 + J/4
E < 3*A/2 + 3*B/2 + C/2 + H - 3*I/2 - J/2
A > -B - C/4 - D/4 + E/4 + 3*G/4
H > I
B > -3*A/4 - C/2 - D/2 - F/2 - G/4 - H/2 + I/2
E < -2*A + B - C + D + F + H - J
B > -A/2 + 3*C/4 + D/4 - F/2 - G/4 - H/4 + I/4 + J/4
A > -3*B/4 - C/2 - D/2 + E/2 - F/4 - G/4 - H/4 + I/2 + 3*J/2
I > J
E < 3*A/2 + B + C/2 + D/2 - F/2
C > -3*A/2 + B - D/2 + 3*E/2 - F/2 + G/2 - H/2 + J
G > H
A > -B - C/4 - D/4 - F/4 - H/2 + 3*I/4 + J/2
E > F
D > E
C > -3*A/2 - 3*B/2 - D/2 + 3*E/2 - F + 3*G/2 - H/2 + J
A > -B - C/2 - D/4 - F/2 - G/4 - H/2 + I/2
Solution:  [7.187499999999997, 3.000000000000004, 4.000000000000005, 7.000000000000024, 7.000000000000016, 7.000000000000008, 9.000000000000021, 9.00000000

In [61]:
rule_1975_2015 = Rule([12,10,8,7,6,5,4,3,2,1], 10)

ESC_classified = dict_to_classes(import_scores_linear_only(1975, 2016), rule_1975_2015)
ESC_classified['1975'].print_result()

nl 152
gb 138
it 115
fr 91
lu 84
ch 77
fi 74
se 72
ie 68
es 53
il 40
mt 32
yu 22
mc 22
be 17
pt 16
de 15
no 11
tr 3



In [13]:
rule_1962 = Rule([3,2,1], 3)
rule_1963 = Rule([5,4,3,2,1], 5)
rule_1964_66 = Rule([5,3,1], 3)

ESC_old = dict_to_classes(import_scores_linear_only(1962, 1967), rule_1964_66)

ESC_old['1962'].print_result()
ESC_old['1963'].print_result()
ESC_old['1964'].print_result()
ESC_old['1965'].print_result()
ESC_old['1966'].print_result()

fr 40
mc 20
lu 17
yu 15
gb 15
de 13
se 6
fi 6
it 4
no 3
ch 3
dk 2

ch 31
dk 31
it 24
gb 15
mc 14
fr 14
at 7
lu 4
be 3
de 1

it 49
gb 17
mc 15
fr 14
lu 14
at 11
fi 9
no 6
dk 4
nl 2
be 2
es 1

lu 32
gb 25
fr 22
at 16
it 15
ie 11
dk 10
ch 8
mc 7
se 6
nl 5
yu 2
no 1
pt 1

at 31
se 16
no 15
ie 14
be 14
ch 12
yu 9
es 9
gb 8
lu 7
de 7
fi 7
pt 6
dk 4
nl 2
fr 1



In [14]:
rule_1975_2015 = Rule([12,10,8,7,6,5,4,3,2,1], 10)
d_jury, d_tele, d_total = import_scores_2015_and_beyond()
ESC_classified_2015 = dicts_to_2015_classes(d_jury, d_tele, d_total, rule_1975_2015)

In [15]:
rule_1962 = Rule([3,2,1], 3)
rule_1963 = Rule([5,4,3,2,1], 5)
rule_1964_66 = Rule([5,3,1], 3)
rule_1975_2015 = Rule([12,10,8,7,6,5,4,3,2,1], 10)
ESC_classified_2015['2019'].print_result(rule_1975_2015)

nl 293
it 271
ru 202
no 190
ch 186
se 171
mk 150
az 148
au 136
is 112
cz 70
cy 49
mt 47
al 47
dk 43
rs 42
si 37
sm 34
fr 33
gr 32
ee 31
es 18
by 15
il 13
de 8



In [16]:
def compute_scores_for_rules(ESC_dict, rules):
    """Takes as input a dict of ESCs and a list of rules.
    Outputs a list with a dict of ESCs for each rule.
    Optionally prints the scores for each rule."""
    
    for rule in rules:
        for contest in ESC_dict:
            print("### " + str(contest) + " normal ###")
            ESC_dict[contest].print_result()
            print("### " + str(contest) + " new rule ###")
            ESC_dict[contest].print_result(rule)
        
    

In [17]:
rules_list = [Rule([3,2,1], 3), Rule([5,4,3,2,1], 5), Rule([5,3,1], 3)]
compute_scores_for_rules(ESC_classified, rules_list)

### 1975 new rule ###
nl 27
gb 22
it 15
fr 11
fi 10
lu 8
ch 6
ie 5
pt 3
il 2
se 2
es 1
mt 1
de 1

### 1976 new rule ###
gb 33
fr 25
it 10
at 7
ch 6
mc 6
be 6
ie 6
il 4
pt 3
gr 1
nl 1

### 1977 new rule ###
gb 22
fr 20
ie 20
mc 10
gr 9
ch 7
be 6
fi 4
nl 3
de 3
il 2
lu 1
it 1

### 1978 new rule ###
il 27
be 17
fr 13
ie 12
mc 11
lu 9
de 8
gr 5
ch 4
es 4
it 4
nl 3
se 2
gb 1

### 1979 new rule ###
il 23
es 20
de 13
fr 13
dk 9
ch 6
gb 6
ie 5
no 4
gr 4
nl 3
it 3
pt 2
lu 2
fi 1

### 1980 new rule ###
ie 25
de 19
nl 15
ch 13
it 10
gb 10
se 5
pt 4
tr 4
lu 3
at 2
be 2
es 1
gr 1

### 1981 new rule ###
ch 21
fr 19
de 18
gb 17
ie 14
cy 7
se 5
es 4
dk 3
yu 3
gr 3
lu 2
il 2
pt 1
be 1

### 1982 new rule ###
de 35
ch 15
il 15
gb 8
cy 8
be 6
lu 4
es 4
se 3
at 3
yu 3
no 2
tr 1
ie 1

### 1983 new rule ###
lu 26
yu 22
il 19
se 16
de 11
gb 6
gr 6
fr 4
nl 3
no 2
at 2
fi 1
be 1
it 1

### 1984 new rule ###
ie 23
se 22
es 12
be 11
dk 10
it 9
fr 6
cy 5
gb 4
ch 3
yu 2
pt 2
tr 2
nl 1
no 1
lu 1

### 1985 new rule ##

fi 7
ch 7
de 6
gb 5
mt 4
gr 3
si 3

### 2015 new rule ###
se 136
ru 102
it 95
be 52
au 48
lv 37
ee 16
az 15
me 13
no 13
al 11
rs 10
ge 9
il 9
gr 7
am 6
lt 5
ro 5
cy 4
si 4
hu 3

### 1975 new rule ###
nl 42
gb 34
it 23
fi 15
fr 15
lu 12
ch 9
ie 8
pt 5
il 3
se 2
es 1
mt 1
de 1

### 1976 new rule ###
gb 52
fr 39
it 15
at 10
ch 9
ie 9
mc 8
be 8
il 5
pt 5
gr 1
nl 1

### 1977 new rule ###
gb 35
fr 31
ie 30
mc 15
gr 14
ch 10
be 9
fi 6
nl 4
de 3
il 3
lu 1
it 1

### 1978 new rule ###
il 41
be 28
fr 18
ie 18
mc 16
lu 15
de 12
gr 7
es 6
ch 5
it 5
nl 5
se 3
gb 1

### 1979 new rule ###
il 36
es 31
de 21
fr 20
dk 14
ch 9
gb 8
ie 7
gr 6
no 5
nl 4
it 3
pt 3
lu 3
fi 1

### 1980 new rule ###
ie 40
de 29
nl 24
ch 20
it 15
gb 14
se 7
tr 6
pt 5
lu 3
at 3
be 3
es 1
gr 1

### 1981 new rule ###
ch 33
fr 30
de 27
gb 24
ie 22
cy 10
se 8
es 6
dk 5
yu 4
gr 4
lu 3
il 2
pt 1
be 1

### 1982 new rule ###
de 56
ch 23
il 23
gb 13
cy 12
be 8
lu 5
es 5
yu 5
at 4
se 3
no 3
tr 1
ie 1

### 1983 new rule ###
lu 40
yu 34
il 2